Diagnostic Derivative Verification
=====

#### Nick Featherstone (Dec 14, 2018)

This notebook provides a check on the derivatives in Rayleigh as computed *at diagnostic output time*.  Those calculations are, in many cases, handled differently than those conducted during normal time-integration.  The latter is best verified through benchmark analysis.   Note that this notebook does not provide an accuracy check.  Instead, it provides **visual verificiation** that the bookeeping of the code is working at output time.  The accuracy of the Rayleigh derivatives is spectral.  The accuracy of those provided by NumPy is second order; some differences are to be expected.   

In the code that follows, the 8 fundamental fields computed within Rayleigh (**v**, T, P, **B**) along with their first and second derivatives (including cross derivatives) are read in from Rayleigh output.  These derivatives are then calculated independently using the code below, and the two results are compared.  To run this check, one Meridional_Slice, one Equatorial_Slice, and one Shell_Slice file are required.  I suggest using and/or modifying the example main_input file found in *input_examples/derivative_verification*.  It is set up to provide the necessary outputs.

In each plot that follows, the NumPy-calculated derivatives are plotting with red circles, and the Rayleigh provided derivatives are plotted with blue lines.   Small errors (such as those due to misplaced factors of sin(theta)) may not be visible via this presentation.  Change the plots/analysis accordingly depending on what you are checking.  



In [ ]:
from rayleigh_diagnostics import Meridional_Slices, Shell_Slices, Equatorial_Slices
from matplotlib import pyplot as plt
import numpy
ms_file = '00100002'  # the Meridional_Slice file
es_file = '00100002'  # the Equatorial_Slice file
ss_file = '00100002'  # the Shell_Slice file
rind = 30  # The r-index along which to examine theta-, phi-, r-theta-, and r-phi derivatives
tind = 30  # The theta-index along which to examine r-, phi, r-theta, and theta-phi derivatives
pind = 30  # The phi-index along which to examine r-, theta-, r-phi, and theta-phi derivatives

# Quantity codes that correspond to **v**, T, P, and **B**, along with the (latexed) names as strings.
vars = [1,2,3,501,502,801,802,803]
names = ['vr', r'v$\theta$', r'v$\phi$', 'T', 'P', 'Br', r'B$\theta$' , r'B$\phi$' ]


Before we begin, let's define a function for computing cross-derivatives (d^2/dxdy)

In [ ]:
def deriv2d(arr,ax0,ax1):
    '''Returns the cross (2nd) derivative of 2-D Numpy array arr, taken first along
        axis 0 and then along axis 1.  
        ax0:  gridpoints for axis 0
        ax1:  gridpoints for axis 1
        ans:  Return value (NumPy ndarray; float64 type)'''
    shp = arr.shape
    nx0 = shp[0]
    nx1 = shp[1]
    tmp = numpy.zeros((nx0,nx1),dtype='float64')
    ans = numpy.zeros((nx0,nx1),dtype='float64')
    for i in range(nx0):
        tmp[i,:] = numpy.gradient(arr[i,:],ax1)
    for i in range(nx1):
        ans[:,i] = numpy.gradient(tmp[:,i],ax0)
    return ans

Radial-derivatives
------
First, we check derivatives in the r-direction using Meridional_Slices

In [ ]:
ms = Meridional_Slices(ms_file)


dvars = [10,11,12,507,508,810,811,812]  # first radial derivative Rayleigh code
ddvars = [55,56,57,537, 538,855,856,857] # 2nd radial derivative Rayleigh code
nvars = len(vars)
radius = ms.radius

sizetuple = (10,10)
fig, ax = plt.subplots(nrows = nvars,ncols = 3, figsize=sizetuple)
for i in range(nvars):
    br = ms.vals[0,tind,:,ms.lut[vars[i]],0]
    dbr = ms.vals[0,tind,:,ms.lut[dvars[i]],0]
    ddbr = ms.vals[0,tind,:,ms.lut[ddvars[i]],0]
    dbcheck = numpy.gradient(br,radius)
    ddbc = numpy.gradient(dbr,radius)

    ax[i][0].plot(radius,br)
    ax[i][1].plot(radius,dbcheck,'ro')
    ax[i][1].plot(radius,dbr)

    ax[i][2].plot(radius,ddbc,'ro')
    ax[i][2].plot(radius,ddbr)
    if (i == 0):
        ax[i][0].set_title('field')
        ax[i][1].set_title(r'd/dr')
        ax[i][2].set_title(r'd$^2$/dr$^2$')

    ax[i][0].set_ylabel(names[i])
plt.tight_layout()
plt.show()

Theta-derivatives
-----
Next, we check derivatives in the theta-direction using Meridional_Slices

In [ ]:

dvars  = [19,20,21,  513, 514,  819,820,821]  # first derivative Rayleigh code
ddvars = [64,65,66,  543, 544,  864,865,866]  # 2nd derivative Rayleigh code
nvars = len(vars)
theta = numpy.arccos(ms.costheta)


sizetuple = (10,10)
fig, ax = plt.subplots(nrows = nvars,ncols = 3, figsize=sizetuple)
for i in range(nvars):
    br = ms.vals[0,:,rind,ms.lut[vars[i]],0]
    dbr = ms.vals[0,:,rind,ms.lut[dvars[i]],0]
    ddbr = ms.vals[0,:,rind,ms.lut[ddvars[i]],0]
    dbcheck = numpy.gradient(br,theta)
    ddbc = numpy.gradient(dbr,theta)

    ax[i][0].plot(theta,br)
    ax[i][1].plot(theta,dbcheck,'ro')
    ax[i][1].plot(theta,dbr)

    ax[i][2].plot(theta,ddbc,'ro')
    ax[i][2].plot(theta,ddbr)
    if (i == 0):
        ax[i][0].set_title('field')
        ax[i][1].set_title(r'd/d$\theta$')
        ax[i][2].set_title(r'd$^2$/d$\theta^2$')

    ax[i][0].set_ylabel(names[i])
plt.tight_layout()
plt.show()


Phi-Derivatives
----
We check phi-derivatives using Shell_Slice output.

In [ ]:
ss = Shell_Slices(ss_file)
theta = numpy.arccos(ss.costheta)

vals = ss.vals[:,:,0,:,0]

nphi = ss.nphi
ntheta = ss.ntheta
phi = numpy.zeros(nphi,dtype='float64')
dphi = 2.0*numpy.pi/nphi
for i in range(nphi):
    phi[i] = dphi*i


In [ ]:
dvars  = [28,29,30,  519, 520,  828,829,830]
ddvars = [73,74,75,  549, 550,  873,874,875]
nvars = len(vars)


sizetuple = (10,10)
fig, ax = plt.subplots(nrows = nvars,ncols = 3, figsize=sizetuple)
for i in range(nvars):
    br   = vals[:, tind, ss.lut[vars[i]]]
    dbr  = vals[:, tind, ss.lut[dvars[i]]]
    ddbr = vals[:, tind, ss.lut[ddvars[i]]]
    dbcheck = numpy.gradient(br,phi)
    ddbc = numpy.gradient(dbr,phi)
    #d2br = ms.vals[0,:,:,2,0]
    #bslice = br[tind,:]
    ax[i][0].plot(phi,br)
    ax[i][1].plot(phi,dbcheck,'ro')
    ax[i][1].plot(phi,dbr)

    ax[i][2].plot(phi,ddbc,'ro')
    ax[i][2].plot(phi,ddbr)
    if (i == 0):
        ax[i][0].set_title('field')
        ax[i][1].set_title(r'd/d$\phi$')
        ax[i][2].set_title(r'd$^2$/d$\phi^2$')

    ax[i][0].set_ylabel(names[i])
plt.tight_layout()
plt.show()

Phi-Theta Derivatives
=======
Check phi-theta derivatives using Shell_Slice output:

In [ ]:


ddvars = [100,101,102,  567, 568,  900,901,902]
nvars = len(vars)


sizetuple = (10,10)
fig, ax = plt.subplots(nrows = nvars,ncols = 2, figsize=sizetuple)

for i in range(nvars):
    b   = vals[:, :, ss.lut[vars[i]]]
    dtp   = vals[:,:,ss.lut[ddvars[i]]]
    dtpc = deriv2d(b,phi,theta)


    dtc  = dtpc[ : , tind] 
    dpc  = dtpc[pind , :] 

    ax[i][0].plot(phi,dtc,'ro')
    ax[i][0].plot(phi,dtp[:,tind])

    ax[i][1].plot(theta,dpc,'ro')
    ax[i][1].plot(theta,dtp[pind,:])

    if (i == 0):
        ax[i][0].set_title(' d2 by dr dtheta  (along phi)')
        ax[i][1].set_title(' d2 by dr dtheta  (along theta)')


    ax[i][0].set_ylabel(names[i])
plt.tight_layout()
plt.show()

R-Theta Derivatives
====
Check the r-theta cross derivatives using Meridional_Slice output:

In [ ]:
vals = ms.vals[0,:,:,:,0]
theta = numpy.arccos(ms.costheta)
radius = ms.radius

ddvars = [82,83,84,  555, 556,  882,883,884]
nvars = len(vars)


sizetuple = (10,10)
fig, ax = plt.subplots(nrows = nvars,ncols = 2, figsize=sizetuple)

for i in range(nvars):

    b   = vals[:, :, ms.lut[vars[i]]]
    dtr   = vals[:,:,ms.lut[ddvars[i]]]
    dtrc = deriv2d(b,theta,radius)


    drc  = dtrc[ tind , :] 
    dtc  = dtrc[: , rind] 


    ax[i][0].plot(radius,drc,'ro')
    ax[i][0].plot(radius,dtr[tind,:])

    ax[i][1].plot(theta,dtc,'ro')
    ax[i][1].plot(theta,dtr[:,rind])

    if (i == 0):
        ax[i][0].set_title(' d2 by dr dtheta  (along r)')
        ax[i][1].set_title(' d2 by dr dtheta  (along theta)')


    ax[i][0].set_ylabel(names[i])
plt.tight_layout()
plt.show()

R-Phi Derivatives
====
Check r-phi derivatives using Equatorial_Slices.  Note that the symmetry about the equator for the Christensen et al. case means that some variables are zero in the equatorial plane, rendering this check moot for those variables.  As long as their other cross-derivatives look OK, and the r-phi derivatives for variables that are non-zero in the equatorial plane look OK, you should assume the derivatives are being computed accurately.  This is because the same routine is used to calculate cross-derivatives for each variable; if r-phi check passes for one value, it should pass for all.

In [ ]:
es = Equatorial_Slices('00100002')
radius = es.radius
phi = es.phi
vals = es.vals[:,:,:,0]
rvars  = [1 ,2 ,3,   501, 502,  801,802,803]
ddvars = [91,92,93,  561, 562,  891,892,893]
nvars = len(rvars)

pind = 38
rind = 22
sizetuple = (10,10)
fig, ax = plt.subplots(nrows = nvars,ncols = 2, figsize=sizetuple)

for i in range(nvars):

    b   = vals[:, :, es.lut[vars[i]]]
    dpr   = vals[:,:,es.lut[ddvars[i]]]
    dprc = deriv2d(b,phi,radius)


    drc  = dprc[ pind , :] 
    dpc  = dprc[: , rind] 


    ax[i][0].plot(radius,drc,'ro')
    ax[i][0].plot(radius,dpr[pind,:])

    ax[i][1].plot(phi,dpc,'ro')
    ax[i][1].plot(phi,dpr[:,rind])

    if (i == 0):
        ax[i][0].set_title(' d2 by dr dphi  (along r)')
        ax[i][1].set_title(' d2 by dr dphi  (along phi)')


    ax[i][0].set_ylabel(names[i])
plt.tight_layout()
plt.show()